# Scrapping Ulasan Minecraft di Playstore

In [ ]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00


In [ ]:
from google_play_scraper import reviews, Sort
import pandas as pd
import csv

In [ ]:
# Mengambil semua ulasan aplikasi Minecraft dari Google Play Store (bahasa Indonesia)
scrapreview, _ = reviews(
    'com.supercell.clashofclans',   # ID aplikasi Minecraft
    lang='eng',                 # Bahasa Indonesia
    country='id',               # Negara Indonesia
    sort=Sort.MOST_RELEVANT,    # Sortir berdasarkan ulasan paling relevan
    count=11000                 # Maksimal jumlah ulasan yang diambil
)

In [ ]:
# Menyimpan hasil scraping ke dalam file CSV
with open('clashofclans.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Rating', 'Date', 'Helpful Count', 'Review'])  # Header kolom
    for review in scrapreview:
        writer.writerow([
            review['score'],
            review['at'],
            review['thumbsUpCount'],
            review['content'],
        ])

In [ ]:
df = pd.read_csv('clashofclans.csv')
df

,Rating,Date,Helpful Count,Review
0,4,2025-03-31 11:27:26,11,"After a several years i played this game, i ha..."
1,3,2025-04-10 11:04:54,2,I love this game. I played this game since 201...
2,4,2025-03-31 13:50:20,2,The fact that this game is now playable on PC ...
3,5,2025-04-06 06:57:31,8,A very tactical game which despite having micr...
4,5,2025-02-12 08:23:26,51,The best game i've ever played!! i don't know ...
...,...,...,...,...
10995,5,2022-04-21 12:22:28,0,Awesome game for sure. A classic for the categ...
10996,5,2020-04-24 19:06:45,97,Real awesome and fun to play keep the good wor...
10997,5,2025-02-09 06:59:23,0,The best game ever infact i can't find the per...
10998,4,2020-05-08 18:45:36,67,It is a brilliant game but there are annoying ...


# Labeling (3 Kelas)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch

In [ ]:
# Load model dan tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
batch_size = 32
reviews = df['Review'].astype(str).tolist()
sentimen_list = []

for i in tqdm(range(0, len(reviews), batch_size)):
    batch_texts = reviews[i:i+batch_size]

    # Tokenisasi batch
    encoded_input = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

    # Prediksi
    with torch.no_grad():
        output = model(**encoded_input)
        scores = output.logits.cpu().numpy()
        for score in scores:
            probs = softmax(score)
            label = np.argmax(probs)
            sentimen_list.append(["Negative", "Neutral", "Positive"][label])

100%|██████████| 344/344 [01:10<00:00,  4.87it/s]


In [ ]:
df['sentimen'] = sentimen_list
df.to_csv("clashofclans_review.csv", index=False)
print("DONE!!!!")

DONE!!!!


In [ ]:
df

,Rating,Date,Helpful Count,Review,sentimen
0,4,2025-03-31 11:27:26,11,"After a several years i played this game, i ha...",Positive
1,3,2025-04-10 11:04:54,2,I love this game. I played this game since 201...,Positive
2,4,2025-03-31 13:50:20,2,The fact that this game is now playable on PC ...,Positive
3,5,2025-04-06 06:57:31,8,A very tactical game which despite having micr...,Positive
4,5,2025-02-12 08:23:26,51,The best game i've ever played!! i don't know ...,Positive
...,...,...,...,...,...
10995,5,2022-04-21 12:22:28,0,Awesome game for sure. A classic for the categ...,Positive
10996,5,2020-04-24 19:06:45,97,Real awesome and fun to play keep the good wor...,Positive
10997,5,2025-02-09 06:59:23,0,The best game ever infact i can't find the per...,Positive
10998,4,2020-05-08 18:45:36,67,It is a brilliant game but there are annoying ...,Negative
